In [1]:
import pandas as pd
import csv
#https://github.com/mayank408/TFIDF/blob/master/TFIDF.ipynb

In [2]:
glencore = pd.read_csv('glencorePR.csv', delimiter = ',')

In [3]:
glencore = glencore.drop(['_source.title', '_score', '_source.doctype', '_type', '_index', '_source.url', '_source.feedurl'], axis = 1)
glencore = glencore.rename(index=str, columns={"_source.title_rss": "title", "_source.teaser_rss": "content"})

In [4]:
contents = []
contents = glencore.content.astype(str)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
contents[0]

'<![CDATA[<p>Glencore plc has today announced that its Board is recommending an aggregate distribution of US$0.20 per share in respect of the 2017 financial year ended 31 December.</p>  <p>Distributions will be declared and paid in U.S. dollars, although Shareholders on the Jersey register will be able to elect to receive their distribution payments in Pounds Sterling, Euros or Swiss Francs. Shareholders who hold shares on the Jersey Register, through the Computershare HK Nominee, will receive their distribution payment in Hong Kong Dollars, converted on the Jersey applicable exchange rate reference date. Shareholders on the Johannesburg register will receive their distributions in South African Rand.</p>  <p>Subject to shareholder approval, the distribution will be made from the capital contribution reserves of the Company in two equal tranches of $0.10 payable in the first six months of the 2018 financial year (H1) and second six months of the 2018 financial year (H2) in accordance w

In [6]:
vectorizer = TfidfVectorizer()
response = vectorizer.fit_transform([c1, c2])

NameError: name 'c1' is not defined

In [5]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/xijiahu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/xijiahu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
import re

In [7]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(contents):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', contents)
    stop_free = " ".join([str(i) for i in cleantext.lower().split() if i not in stop])
    stop_free.replace('nan',' ')
#remove "nan" from text
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in contents]

In [15]:
c1 = doc_clean[0]
c2 = doc_clean[1]

In [11]:
wordSet = set(c1).union(set(c2))

In [12]:
wordDictA = dict.fromkeys(wordSet, 0) 
wordDictB = dict.fromkeys(wordSet, 0)

In [13]:
for word in c1:
    wordDictA[word]+=1
    
for word in c2:
    wordDictB[word]+=1

In [14]:
pd.DataFrame([wordDictA, wordDictB])

,010,04,10,12,13,14,146000,150,155000,2,...,websitenbspwwwnyrstarcom,wednesday,word,work,working,world,worldrsquos,wwwglencorecom,year,zinc
0,1,1,2,2,3,1,1,1,0,2,...,0,6,1,1,0,0,1,1,3,0
1,0,0,0,0,0,0,0,1,1,0,...,1,0,1,1,2,3,1,0,0,3


In [15]:
def  computeTF(wordDict, doc_clean):
    tfDict = {}
    docCount = len(doc_clean)
    for word, count in wordDict.items():
        tfDict[word] = count/float(docCount)
    return tfDict

In [16]:
tfBowA = computeTF(wordDictA, c1)
tfBowB = computeTF(wordDictB, c2)

In [17]:
sorted(tfBowA.items(), key=lambda x:x[1])

[('includes', 0.0),
 ('credit', 0.0),
 ('isa', 0.0),
 ('headquartered', 0.0),
 ('maintenance', 0.0),
 ('premised', 0.0),
 ('changing', 0.0),
 ('coupled', 0.0),
 ('metallurgy', 0.0),
 ('isasmelttmnbspprocess', 0.0),
 ('incorporated', 0.0),
 ('implementation', 0.0),
 ('essential', 0.0),
 ('certain', 0.0),
 ('industrialisation', 0.0),
 ('kazzinc', 0.0),
 ('australia', 0.0),
 ('building', 0.0),
 ('port', 0.0),
 ('visitnbspwwwkazzinccom', 0.0),
 ('providing', 0.0),
 ('symbol', 0.0),
 ('2462nbsp', 0.0),
 ('switzerland', 0.0),
 ('flexibility', 0.0),
 ('multimetals', 0.0),
 ('pioneered', 0.0),
 ('follows', 0.0),
 ('recovery', 0.0),
 ('base', 0.0),
 ('develop', 0.0),
 ('partnership', 0.0),
 ('many', 0.0),
 ('four', 0.0),
 ('announces', 0.0),
 ('lead', 0.0),
 ('engineering', 0.0),
 ('purest', 0.0),
 ('approach', 0.0),
 ('selection', 0.0),
 ('visit', 0.0),
 ('training', 0.0),
 ('websitenbspwwwnyrstarcom', 0.0),
 ('installation', 0.0),
 ('part', 0.0),
 ('ramp', 0.0),
 ('design', 0.0),
 ('stage', 0

In [18]:
def  computeIDF (docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict

In [19]:
idfs = computeIDF([wordDictA, wordDictB])

In [20]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [21]:
tfidfBowA = computeTFIDF(tfBowA, idfs)
tfidfBowB = computeTFIDF(tfBowB, idfs)

In [22]:
import pandas as pd
pd.DataFrame([tfidfBowA, tfidfBowB])

,010,04,10,12,13,14,146000,150,155000,2,...,websitenbspwwwnyrstarcom,wednesday,word,work,working,world,worldrsquos,wwwglencorecom,year,zinc
0,0.000465,0.000465,0.000931,0.000931,0.001396,0.000465,0.000465,0.0,0.000000,0.000931,...,0.000000,0.002792,0.0,0.0,0.000000,0.000000,0.0,0.000465,0.001396,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000649,0.000000,...,0.000649,0.000000,0.0,0.0,0.001298,0.001946,0.0,0.000000,0.000000,0.001946


In [11]:
from  sklearn.feature_extraction.text  import TfidfVectorizer
#the second methods to calculate tfidf

In [13]:
tfidf  =  TfidfVectorizer ()

In [21]:
response = tfidf.fit_transform(c1, c2)

In [19]:
feature_names = tfidf.get_feature_names()
for col in response.nonzero()[1]:
    print(feature_names[col], ' - ', response[0, col])

glencore  -  1.0
plc  -  0.0
today  -  0.0
announced  -  0.0
board  -  0.0
recommending  -  0.0
aggregate  -  0.0
distribution  -  0.0
us020  -  0.0
per  -  0.0
share  -  0.0
respect  -  0.0
2017  -  0.0
financial  -  0.0
year  -  0.0
ended  -  0.0
31  -  0.0
december  -  0.0
distribution  -  0.0
declared  -  0.0
paid  -  0.0
dollar  -  0.0
although  -  0.0
shareholder  -  0.0
jersey  -  0.0
register  -  0.0
able  -  0.0
elect  -  0.0
receive  -  0.0
distribution  -  0.0
payment  -  0.0
pound  -  0.0
sterling  -  0.0
euro  -  0.0
swiss  -  0.0
franc  -  0.0
shareholder  -  0.0
hold  -  0.0
share  -  0.0
jersey  -  0.0
register  -  0.0
computershare  -  0.0
hk  -  0.0
nominee  -  0.0
receive  -  0.0
distribution  -  0.0
payment  -  0.0
hong  -  0.0
kong  -  0.0
dollar  -  0.0
converted  -  0.0
jersey  -  0.0
applicable  -  0.0
exchange  -  0.0
rate  -  0.0
reference  -  0.0
date  -  0.0
shareholder  -  0.0
johannesburg  -  0.0
register  -  0.0
receive  -  0.0
distribution  -  0.0
south 